In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [167]:
from urllib.request import urlopen
from urllib.parse import urljoin

In [168]:
from selenium import webdriver

import requests; from urllib.parse import urlparse

import time
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

### URL가져오기

In [169]:
url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=SteadySeller&BranchType=1&CID=170&page=1&cnt=100&SortOrder=1'

page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

In [170]:
book_name = []
url_name = []

list_soup = soup.find_all('div', 'ss_book_list')

for i,item in enumerate(list_soup) :
    if i%2 == 0:
        book_name.append(item('b')[0].get_text())
        url_name.append(urljoin(url, item.find(class_ = 'bo3')['href']))

In [171]:
len(book_name), len(url_name)

(50, 50)

In [205]:
data = {'Book' : book_name,
        'URL' : url_name}
d_f = pd.DataFrame(data)
d_f.head()

,Book,URL
0,부의 추월차선,https://www.aladin.co.kr/shop/wproduct.aspx?It...
1,전설로 떠나는 월가의 영웅 (2017 최신개정판),https://www.aladin.co.kr/shop/wproduct.aspx?It...
2,부자 아빠 가난한 아빠 1 (20주년 특별 기념판),https://www.aladin.co.kr/shop/wproduct.aspx?It...
3,주식 공부 5일 완성,https://www.aladin.co.kr/shop/wproduct.aspx?It...
4,"돈, 뜨겁게 사랑하고 차갑게 다루어라",https://www.aladin.co.kr/shop/wproduct.aspx?It...


### Selenium이용하여 정보 크롤링

In [317]:
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get(d_f['URL'][0])

**열린 크롬페이지를 스크롤 다운해줍니다.**

In [318]:
driver.find_element_by_xpath('''//*[@id="Underline3_more"]/a''').click()

In [319]:
html = driver.page_source
soup = BeautifulSoup(html)

#### 책속에서 가져오기

In [320]:
tmp = soup.find_all('div', 'Ere_prod_mconts_box')

In [324]:
for i in range(10) :
    tmp_tmp = tmp[i]
    
    try:
        if tmp_tmp.find('div', 'Ere_prod_mconts_LL').get_text() == '책속에서' :
            first_sentence = '첫문장 : ' + tmp_tmp.find('div', 'Fsen_conts').get_text()
        
            senten = []

            senten.append(first_sentence)

            for i in range(10) :
                senten.append(tmp_tmp.find('span', id = 'u3_{}_more'.format(i+1)).get_text())
            
            break;
            
    except:
        pass;

In [325]:
data = {'문장들' : senten}
df = pd.DataFrame(data)
df

,문장들
0,"첫문장 : 부자가 되는 것이 목표인 당신이 '천천히 부자 되기' 전략을 선택했다면,..."
1,P. 64 부는 모호한 개념이 아니다. 내 인생에서 가장 행복한 순간은 진정한 부...
2,P. 100 당신은 자유를 사기 위해 자유를 팔고 있다. 직장에서 돈을 벌려면 반...
3,P. 262 영향력의 법칙에 의하면 수백만 달러를 벌기 위해서는 수백만 명에게 영...
4,바로 그 때 나는 내가 이룬 것이 무엇인지 깨달았다. 그것이 바로 추월 차선이었다....
5,추월차선 여행을 성공적으로 마친 사람들은 거친 길 위에서 살고 죽기를 두려워하지 않...
6,영화 <멋진 인행...>의 마지막 장면은 다음과 같은 교훈을 던져 준다. ˝기억하라...
7,운을 이해하려면 더 높은 확률이 기대되는 과정에 참여하라. 운은 실제로 게임에 참여...
8,생각 없이 행동부터 한 일의 방식 때문에 아주르의 인생은 고통스러워졌다. 아주르는 ...
9,- 저축은 자유를 위해 투쟁할 군대를 모으는 방법이다.- 부자들은 복리를 목돈에 적...


#### 연령별, 성별 분포 가져오기

In [192]:
what = soup.find('div', 'Ere_prod_graphwrap_a')

In [193]:
what_what = what.find('div', 'analysis_box')

In [195]:
what_woman = what_what.find_all('td', 'tb_woman')
what_man = what_what.find_all('td', 'tb_man')
what_age = what_what.find_all('td', 'tb_tit')

In [196]:
buyer_woman = []
buyer_man = []
buyer_age = []

for i in range(len(what_what.find_all('td', 'tb_woman'))):
    buyer_woman.append(what_woman[i].find('div', 'per').get_text())
    buyer_man.append(what_man[i].find('div', 'per').get_text())
    buyer_age.append(what_age[i].get_text())

In [197]:
data = {'연령' : buyer_age,
        '남자' : buyer_man,
        '여자' : buyer_woman}
df = pd.DataFrame(data)
df

,연령,남자,여자
0,10대,0.4%,0.3%
1,20대,12.9%,9.5%
2,30대,18.4%,18.0%
3,40대,12.7%,15.9%
4,50대,4.9%,5.2%
5,60대,1.1%,0.7%


#### 평점 가져오기

In [241]:
wtd = soup.find('div', 'star_box')

In [242]:
wtd_star = wtd.find('div', 'score_box').get_text().replace(' ', '')

In [243]:
wtd_star

'9.4'

In [200]:
wtd_wtd = wtd.find_all('td', 'tb_right')

In [201]:
wtd_list = wtd_wtd[0].get_text().replace(' ', '')

In [202]:
score_list = []
wtd_list = []
total_score = []

for i in range(len(wtd_wtd)):
    score_list.append('{}점'.format(5-i))
    wtd_list.append(wtd_wtd[i].get_text().replace(' ', ''))
    total_score.append(wtd_star)

In [203]:
data = { '별점' : score_list,
         '점수' : wtd_list,
         '총점' : total_score}
df = pd.DataFrame(data)
df

,별점,점수,총점
0,5점,46.5%,8.2
1,4점,29.7%,8.2
2,3점,16.8%,8.2
3,2점,2.0%,8.2
4,1점,5.0%,8.2


### 함수정리

In [326]:
# 책속에서
def inbook(soup) :
    tmp = soup.find_all('div', 'Ere_prod_mconts_box')
    
    for i in range(10) :
        tmp_tmp = tmp[i]
    
        try :
            if tmp_tmp.find('div', 'Ere_prod_mconts_LL').get_text() == '책속에서' :
                first_sentence = '첫문장 : ' + tmp_tmp.find('div', 'Fsen_conts').get_text()
        
                senten = []

                senten.append(first_sentence)

                for i in range(10) :
                    senten.append(tmp_tmp.find('span', id = 'u3_{}_more'.format(i+1)).get_text())
            
                break;
                
        except :
            pass;
            
    data = {'문장들' : senten}
    df = pd.DataFrame(data)
    
    return df

In [247]:
inbook(soup)

,문장들
0,첫문장 : 주식투자 능력은 유전인자로 좌우되지 않는다.
1,P. 66~67 * 만약 스바루 자동차를 살 때 스바루 주식도 함께 샀다면 그 소...
2,P. 179~180 * 어떤 경로로 종목을 발굴하더라도 성급하게 주식을 매수해서는...
3,P. 395 * 투자 자문계의 두 파벌 사이에서는 오랫동안 논쟁이 이어져 왔다. ...
4,다시말해 종목의 수에 얽매이지 말고 종목의 내용을 살펴라. 단순히 많은 기업에 투자...
5,"나는 주식을 매수하기 전에 이 주식에 흥미를 느끼는 이유, 이 회사가 성공하기 위해..."
6,"기본에 바탕을 두고 조심스럽게 가지치기와 교체 매매를 하면, 실적을 향상시킬 수 있..."
7,새로운 아이디어에 항상 마음을 열어 놓으라. - Investor우기ya
8,마음에 든다고 해서 ‘모든 여자에게 입맞춤‘을 할 수는 없다. 나도 10루타 종목을...
9,주말은 물론이고 주중에도 온 나라가 금방이라도 무너질 듯 겅정하는 소리를 듣는다. ...


In [248]:
# 연령, 성별 분포
def dist(soup) :
    what = soup.find('div', 'Ere_prod_graphwrap_a')
    
    what_what = what.find('div', 'analysis_box')
    
    what_woman = what_what.find_all('td', 'tb_woman')
    what_man = what_what.find_all('td', 'tb_man')
    what_age = what_what.find_all('td', 'tb_tit')
    
    buyer_woman = []
    buyer_man = []
    buyer_age = []
    
    for i in range(len(what_what.find_all('td', 'tb_woman'))) :
        buyer_woman.append(what_woman[i].find('div', 'per').get_text())
        buyer_man.append(what_man[i].find('div', 'per').get_text())
        buyer_age.append(what_age[i].get_text())
        
    data = {'연령' : buyer_age,
            '남자' : buyer_man,
            '여자' : buyer_woman}
    df = pd.DataFrame(data)
    
    return df

In [249]:
dist(soup)

,연령,남자,여자
0,10대,0.5%,0.1%
1,20대,19.8%,6.3%
2,30대,22.8%,12.8%
3,40대,13.7%,11.6%
4,50대,5.7%,5.0%
5,60대,1.2%,0.5%


In [250]:
# 평점
def score(soup) :
    wtd = soup.find('div', 'star_box')
    
    wtd_star = wtd.find('div', 'score_box').get_text().replace(' ', '')
    
    wtd_wtd = wtd.find_all('td', 'tb_right')
    
    score_list = []
    wtd_list = []
    total_score = []

    for i in range(len(wtd_wtd)):
        score_list.append('{}점'.format(5-i))
        wtd_list.append(wtd_wtd[i].get_text().replace(' ', ''))
        total_score.append(wtd_star)
        
    data = { '별점' : score_list,
             '점수' : wtd_list,
             '총점' : total_score}
    df = pd.DataFrame(data)
    
    
    return df

In [251]:
score(soup)

,별점,점수,총점
0,5점,74.4%,9.4
1,4점,20.5%,9.4
2,3점,5.1%,9.4
3,2점,0%,9.4
4,1점,0%,9.4


### 크롤링하기

In [353]:
driver = webdriver.Chrome('../driver/chromedriver.exe')
driver.get(d_f['URL'][1])

In [343]:
SCROLL_PAUSE_SEC = 1

last_height = driver.execute_script("return document.body.scrollHeight")

In [344]:
iterate = [10, 100, 1000, 2000, 3000, 4000, 5000, 6000, 2000, 4000]

for i in iterate :
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, {});".format(i))
        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [347]:
# ActionChains 를 사용하기 위해서.
from selenium.webdriver import ActionChains

# id가 something 인 element 를 찾음
some_tag = driver.find_element_by_id('Underline3_more')

# somthing element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(some_tag).perform()

In [354]:
driver.find_element_by_xpath('''//*[@id="Underline3_more"]/a''').click()

In [355]:
html = driver.page_source
soup = BeautifulSoup(html)

In [356]:
inbook(soup)

,문장들
0,첫문장 : 주식투자 능력은 유전인자로 좌우되지 않는다.
1,P. 66~67 * 만약 스바루 자동차를 살 때 스바루 주식도 함께 샀다면 그 소...
2,P. 179~180 * 어떤 경로로 종목을 발굴하더라도 성급하게 주식을 매수해서는...
3,P. 395 * 투자 자문계의 두 파벌 사이에서는 오랫동안 논쟁이 이어져 왔다. ...
4,다시말해 종목의 수에 얽매이지 말고 종목의 내용을 살펴라. 단순히 많은 기업에 투자...
5,"나는 주식을 매수하기 전에 이 주식에 흥미를 느끼는 이유, 이 회사가 성공하기 위해..."
6,"기본에 바탕을 두고 조심스럽게 가지치기와 교체 매매를 하면, 실적을 향상시킬 수 있..."
7,새로운 아이디어에 항상 마음을 열어 놓으라. - Investor우기ya
8,마음에 든다고 해서 ‘모든 여자에게 입맞춤‘을 할 수는 없다. 나도 10루타 종목을...
9,주말은 물론이고 주중에도 온 나라가 금방이라도 무너질 듯 겅정하는 소리를 듣는다. ...


In [357]:
dist(soup)

,연령,남자,여자
0,10대,0.5%,0.1%
1,20대,19.8%,6.3%
2,30대,22.8%,12.8%
3,40대,13.7%,11.6%
4,50대,5.7%,5.0%
5,60대,1.2%,0.5%


In [358]:
score(soup)

,별점,점수,총점
0,5점,74.4%,9.4
1,4점,20.5%,9.4
2,3점,5.1%,9.4
3,2점,0%,9.4
4,1점,0%,9.4
